## API Featching for CQC rating in Dudley Care Homes

In [1]:
import requests
import json

In [2]:
# Base URL for the CQC API
base_url = "https://api.service.cqc.org.uk/public/v1/locations"

# Parameters for the API request
params = {
    "localAuthority": "Dudley",  
    "careHome": "Y",            
    "perPage": 100               
}

# Subscription key for authentication
subscription_key = "6c0491618d764af0a892e9a51c9666ce"

# Headers for the API request
headers = {
    # User-Agent header to avoid getting blocked
    "User-Agent": "Mozilla/5.0",  
    # Subscription key for authentication
    "Ocp-Apim-Subscription-Key": subscription_key  
}

def fetch_data(url, params, headers):
    """
    Fetch data from the API.
    """
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        # Return the JSON response if successful
        return response.json()  
    return None

def fetch_care_home_ratings(location_id, headers):
    """
    Fetch overall ratings for a care home by its location ID.
    """
    url = f"https://api.service.cqc.org.uk/public/v1/locations/{location_id}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        rating = data.get('currentRatings', {}).get('overall', {}).get('rating')
        return rating
    return None

# Fetch data from the CQC API
data = fetch_data(base_url, params, headers)

# Check if data was successfully fetched
if data:
    # Process and print the data as needed
    for location in data.get('locations', []):
        name = location.get('locationName')
        location_id = location.get('locationId')
        rating = fetch_care_home_ratings(location_id, headers)
        if rating is not None:
            print(f"Care Home: {name}, Rating: {rating}")


Care Home: Cygnet House, Rating: Good
Care Home: 60 Olive Lane, Rating: Good
Care Home: Arcare for Forte Limited, Rating: Good
Care Home: Hollybush House Nursing Home, Rating: Good
Care Home: Woodview House Nursing Home, Rating: Requires improvement
Care Home: Commonside Care Limited - 73 Commonside, Rating: Good
Care Home: Stanford House, Rating: Requires improvement
Care Home: Tudor Manor, Rating: Inadequate
Care Home: Attwood Street, Rating: Good
Care Home: Langstone Road, Rating: No published rating
Care Home: Stickley Lane, Rating: Requires improvement
Care Home: Edmore House Residential Home, Rating: Good
Care Home: Belvidere Residential Care Home, Rating: Requires improvement
Care Home: Woodlands Gate Rest Home, Rating: Good
Care Home: Two Gates House, Rating: Good
Care Home: Eve House, Rating: No published rating
Care Home: Nelson House, Rating: Requires improvement
Care Home: Mr and Mrs O'Donnell, Rating: Inadequate
Care Home: Halas Homes, Rating: Good
Care Home: Castlewellan 

In [3]:
# Iterate over the locations array and extract information
for location in data.get('locations', []):
    name = location.get('locationName')
    postal_code = location.get('postalCode')
    rating = location.get('rating')  # Assuming that 'rating' is a key in the location dictionary
    print(f"Care Home: {name}, Postal Code: {postal_code}, Rating: {rating}")


Care Home: Cygnet House, Postal Code: DY8 3UU, Rating: None
Care Home: Parkfield Grange, Postal Code: DY8 1QH, Rating: None
Care Home: 60 Olive Lane, Postal Code: B62 8LZ, Rating: None
Care Home: Arcare for Forte Limited, Postal Code: WV14 9QB, Rating: None
Care Home: Hollybush House Nursing Home, Postal Code: DY8 4HZ, Rating: None
Care Home: Woodview House Nursing Home, Postal Code: B63 1EN, Rating: None
Care Home: Commonside Care Limited - 73 Commonside, Postal Code: DY5 4AJ, Rating: None
Care Home: Stanford House, Postal Code: DY3 1ST, Rating: None
Care Home: Tudor Manor, Postal Code: DY8 3XF, Rating: None
Care Home: Attwood Street, Postal Code: B63 3UE, Rating: None
Care Home: Langstone Road, Postal Code: DY1 2NJ, Rating: None
Care Home: Stickley Lane, Postal Code: DY3 2JQ, Rating: None
Care Home: Edmore House Residential Home, Postal Code: DY2 7TB, Rating: None
Care Home: Belvidere Residential Care Home, Postal Code: DY1 2DH, Rating: None
Care Home: Woodlands Gate Rest Home, Posta

In [26]:
import pandas as pd

data_list = []

if data:
    for location in data.get('locations', []):
        name = location.get('locationName')
        location_id = location.get('locationId')
        postal_code = location.get('postalCode')
        rating = fetch_care_home_ratings(location_id, headers)

        
        if rating is not None and postal_code.startswith('D'):
            data_list.append({'Care Home': name, 'Postal Code': postal_code, 'Rating': rating})

# Convert the list of dictionaries to a DataFrame
ratings_df = pd.DataFrame(data_list)

# Print the first 50 entries of the DataFrame
ratings_df.head(50)

,Care Home,Postal Code,Rating
0,Cygnet House,DY8 3UU,Good
1,Hollybush House Nursing Home,DY8 4HZ,Good
2,Commonside Care Limited - 73 Commonside,DY5 4AJ,Good
3,Stanford House,DY3 1ST,Requires improvement
4,Tudor Manor,DY8 3XF,Inadequate
5,Langstone Road,DY1 2NJ,No published rating
6,Stickley Lane,DY3 2JQ,Requires improvement
7,Edmore House Residential Home,DY2 7TB,Good
8,Belvidere Residential Care Home,DY1 2DH,Requires improvement
9,Woodlands Gate Rest Home,DY9 0RS,Good


In [25]:

# Load the CSV file
file_path = "C:/Users/reach/Documents/Class/Project 3/Project-3-Care-Home-Team-5/data/nhomes_cleaned_coordinates.csv"
nhomes_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to understand its structure
nhomes_df.head()

,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \nNursing,Capacity,Post Code,Care Home Name,Min Age,Max Age,Coordinates,Latitude,Longitude
0,Abbeygate Care Centre 2 Leys Road Brockmoor Br...,abbeygatecare1@gmail.com,01384 571295,Age 65+,Dementia \nMental Health Condition \nOld Age \...,No,17.0,DY5 3UR,Abbeygate Care,65.0,NaN,"[52.4857514, -2.1341733]",52.485751,-2.134173
1,Abbeymere 12 Eggington Road Wollaston Stourbri...,abbeymere@karelink.co.uk,01384 395195,Ages 65+,Dementia \nMental Health Condition \nOld Age \...,No,18.0,DY8 2QJ,Abbeymere,65.0,NaN,"[52.4584, -2.16295]",52.458400,-2.162950
2,Allenbrook Nursing Home 209 Spies Lane Halesow...,manager@allenbrooknursing\nhome.co.uk,0121 422 5844,Ages 55+,Dementia \nMental Health Condition \nNo Medica...,Yes,36.0,B62 9SJ,Allenbrook Nursing,55.0,NaN,"[52.4613003, -2.0139452]",52.461300,-2.013945
3,Amberley Care Home 481-483 Stourbridge Road Ha...,amberleycarehome@hotmail\n.co.uk,01384 482365,Ages 65+,Dementia \nOld Age,No,25.0,DY5 1LB,Amberley Care Home,65.0,NaN,"[52.4927718, -2.1164204]",52.492772,-2.116420
4,Arcare For Forte 440 Birmingham New Road Bilst...,ksharma@arcarehomes.co.u\nk,01902 880108,18+,Learning Disability \nMental Health Condition ...,No,9.0,WV14 9QB,Arcare For Forte,18.0,NaN,"[52.5481562, -2.094362]",52.548156,-2.094362


In [16]:
# Filter for postal codes starting with 'DY'
dy_nh_df = nhomes_df[nhomes_df['Post Code'].str.startswith('DY', na=False)].copy()

# Correct way to ensure changes are made directly in the DataFrame
dy_nh_df.loc[:, 'Post Code'] = dy_nh_df['Post Code'].str.strip()

# Merge the DataFrames based on postal codes starting with 'DY'
matched_df = pd.merge(dy_nh_df, ratings_df, left_on='Post Code', right_on='Postal Code', how='inner')

# Display the merged DataFrame with matched ratings
matched_df[['Care Home Name', 'Post Code', 'Rating']]


,Care Home Name,Post Code,Rating
0,Abbeygate Care,DY5 3UR,No published rating
1,Ashgrove Nursing,DY2 0DA,No published rating
2,Ashgrove Nursing,DY2 0DA,Requires improvement
3,Belvidere,DY1 2DH,Requires improvement
4,Brambles Rest Home,DY8 4JA,Requires improvement
5,Broadway Halls,DY1 3EA,Requires improvement
6,Camelot Rest Home,DY1 2ER,Requires improvement
7,Camelot Rest Home,DY1 2ER,No published rating
8,Holly Hall,DY1 2ER,Requires improvement
9,Holly Hall,DY1 2ER,No published rating


In [20]:
# Ensure postal code columns in both DataFrames are properly formatted
nhomes_df['Post Code'] = nhomes_df['Post Code'].str.strip()
ratings_df['Postal Code'] = ratings_df['Postal Code'].str.strip()

# Filter nhomes_df for postal codes starting with 'DY' and merge with ratings
filtered_nh_df = nhomes_df[nhomes_df['Post Code'].str.startswith('DY', na=False)].copy()
filtered_nh_df['Post Code'] = filtered_nh_df['Post Code'].str.strip()

# Merge the filtered DataFrame with ratings
merged_df = pd.merge(filtered_nh_df, ratings_df, left_on='Post Code', right_on='Postal Code', how='inner')

# To include ratings in the original nhomes_df
final_df = pd.merge(nhomes_df, merged_df[['Post Code', 'Rating']], on='Post Code', how='left')

# Display the merged DataFrame
final_df.tail(5)


,Name & Address,Email,Telephone No,Age Range,CQC Registered For,Provide \nNursing,Capacity,Post Code,Care Home Name,Min Age,Max Age,Coordinates,Latitude,Longitude,Rating
111,Woodlands Gate Rest Home 12 Dingle Road Pedmor...,woodlands@sclcare.co.uk,01562 885546,Ages 60+,Old Age \nPhysical Disability \nSensory Impair...,No,20.0,DY9 0RS,Woodlands Gate Rest,60.0,NaN,"[52.4434178, -2.1320989]",52.443418,-2.132099,Good
112,Woodview House Nursing Home Waugh Drive Hagley...,woodview@shaw.co.uk,0121 585 6440,Ages 65+,C\nMental Health Condition \nOld Age \nCG Comm...,Yes,24.0,B63 1EN,Woodview House,65.0,NaN,"[52.4356855, -2.0820804]",52.435685,-2.082080,NaN
113,Wordsley Hall Mill Street Wordsley Stourbridge...,manager.wordsleyhall@min\nstercaregroup.co.uk,01384 571606,Ages 65+,"Old Age, Dementia",No,41.0,DY8 5SX,Wordsley Hall,65.0,NaN,"[52.4780241, -2.1562237]",52.478024,-2.156224,Good
114,Wychbury Care Home 350-352 Hagley Road Pedmore...,daniel@wychcare.com,01562 885106,Ages 65+,", Old Age",No,42.0,DY9 0QY,Wychbury Care Home,65.0,NaN,"[52.4327421, -2.1301692]",52.432742,-2.130169,Good
115,Wyncroft House 16 Moss Grove Kingswinford DY6 9HU,manager@wyncrofthouse.c\no.uk,01384 291688,Ages 65+,"Old Age, Dementia",Yes,38.0,DY6 9HU,Wyncroft House,65.0,NaN,"[52.5025546, -2.1702452]",52.502555,-2.170245,Requires improvement


In [23]:
# Specify the path where you want to save the CSV file
output_file_path = "C:/Users/reach/Documents/Class/Project 3/Project-3-Care-Home-Team-5/data/final_nh_data_with_ratings.csv"

# Save the DataFrame to a CSV file
final_df.to_csv(output_file_path, index=False)

print("Data has been successfully exported to:", output_file_path)


Data has been successfully exported to: C:/Users/reach/Documents/Class/Project 3/Project-3-Care-Home-Team-5/data/final_nh_data_with_ratings.csv
